In [1]:
# import wandb
from mmcv import Config
from mmseg.datasets import build_dataset

from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.datasets import (build_dataloader, build_dataset)
import mmcv

In [2]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset
import os.path as osp
import numpy as np
from PIL import Image
import os
from mmseg.apis import set_random_seed

data_root = '/opt/ml/input/last'
classes = ('Background','General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')
palette = [[0,0,0],[192,0,128],[0,128,192],[0,128,64],[128,0,0],[64,0,128],[64,0,192],[192,128,64],[192,192,128],[64,64,128],[128,0,192]]

In [3]:
# model 이름
model_path = './mmsegmentation/deeplab3/configs/_boost_/train_model'
model_name = 'deeplabv3_m-v2-d8_512x1024_80k_cityscapes.py'
cfg = Config.fromfile(os.path.join(model_path,model_name))

In [4]:
# cfg.norm_cfg = dict(type='BN', requires_grad=True)
# cfg.model.backbone.norm_cfg = cfg.norm_cfg
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# # cfg.model.neck.norm_cfg = cfg.norm_cfg
# cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# # cfg.model.decode_head.num_classes = 11
# # cfg.model.auxiliary_head.num_classes = 11

# # data root
# cfg.data_root = data_root

# batch size
cfg.data.samples_per_gpu = 6
cfg.data.workers_per_gpu=8

# # cfg.data.train.data_root = cfg.data_root
# # cfg.data.train.img_dir = 'images'
# # cfg.data.train.ann_dir = 'annotations'
# # cfg.data.train.split = data_root+'/splits/train.txt'


# cfg.data.val.data_root = cfg.data_root
# cfg.data.val.img_dir = 'images'
# cfg.data.val.ann_dir = 'annotations'
# cfg.data.val.split = data_root+'/splits/valid.txt'


# # where checkpoints saved
cfg.work_dir = './work_dirs/deeplab3'

cfg.runner.max_epochs = 10 # max epoch

cfg.log_config.interval = 200 # train 정보 출력 간격
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=6, interval=2) # max_keep_ckpts : pht 파일 최대 저장 갯수 , interval : pth 저장 간격
# cfg.optimizer_config= dict(type='Fp16OptimizerHook', loss_scale='dynamic',grad_clip = dict(max_norm=35, norm_type=2))
# # Set seed to facitate reproducing the result
cfg.seed = 21
# set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]


In [5]:
datasets = build_dataset(cfg.data.train)


2022-05-11 19:52:16,221 - mmseg - INFO - Loaded 4417 images


In [6]:
model = build_segmentor(cfg.model)
model.init_weights()

In [ ]:
train_segmentor(model, datasets, cfg, distributed=False, validate=False,meta=dict())
# 모델 학습
